In [303]:
import pandas as pd

import warnings
warnings.filterwarnings(action='ignore')

In [421]:
# 필요데이터만 뽑아서 df불러오기
def read_data(year,month):
    if month < 10:
        month = '0' + str(month)
    data_name = 'data/' + str(year) + str(month) + '.csv'
    df = pd.read_csv(data_name, encoding='cp949', header=None)
    df = df.iloc[:, [0,2,5,6]]
    weight_name = str(year) + '-' + str(month) + '중량'
    price_name = str(year) + '-' + str(month) + '금액'
    df.columns = ['AGCODE','품목', weight_name, price_name]
    
    # 2016~2019년 데이터에서 품목컬럼에서 NAN값 존재하여 다음 코드 실행에 오류발생
    # -> 확인결과 AGCODE : 169220450 값으로 제거대상이므로 삭제
    df.dropna(inplace=True)

    # 농산물 데이터만 가져오기 : ASCODE -> 1로 시작
    df.AGCODE = df.AGCODE.astype(str)
    df = df[df.AGCODE.str.startswith('1')]
    
        
    # 중분류 데이터 가져오기
    df = df[df.AGCODE.str.len()==3]


    # 필요없는 데이터 제거
    drop_items = ['114', '119', '131', '132', '134', '137', '139', '141', '142', '144', '151', '152', '153', '155', '156',
                 '158', '159', '161', '169', '170', '171', '174']
    for item in drop_items:
        df = df.drop(df[df.AGCODE.str.startswith(item)].index)

    # 중량, 금액 타입 변환
    df[weight_name] = df[weight_name].str.replace(',','')
    df[price_name] = df[price_name].str.replace(',','')
    df[[weight_name, price_name]] = df[[weight_name, price_name]].apply(pd.to_numeric)
    
    return df

In [405]:
# 년도별로 df 병합시켜주는 함수
def merge_data(year):
    df = read_data(year,1)
    for i in range(2,13):
        df2 = read_data(year,i)
        df = pd.merge(df, df2, how = 'outer')
    return df

In [432]:
# 품목 이름 정리 ( -계 제거 ) 함수
def substr(df):
    df['품목'] = df['품목'].str[:-2]
    return df

### 코드
- 111 : 곡류  
- 112 : 서류  
- 113 : 두류  
- 115 : 채유종실  
- 116 : 과실류  
- 121 : 채소류  
- 123 : 버섯류  
- 143 : 소스류  
- 157 : 주류  
- 162 : 면류  

In [429]:
# 2016 ~ 2020년 df 생성
df_2020 = substr(merge_data(2020))
df_2019 = substr(merge_data(2019))
df_2018 = substr(merge_data(2018))
df_2017 = substr(merge_data(2017))
df_2016 = substr(merge_data(2016))

In [439]:
df_2020.to_csv('data/농산물_df_2020.csv', index = False)
df_2019.to_csv('data/농산물_df_2019.csv', index = False)
df_2018.to_csv('data/농산물_df_2018.csv', index = False)
df_2017.to_csv('data/농산물_df_2017.csv', index = False)
df_2016.to_csv('data/농산물_df_2016.csv', index = False)